In [1]:
import breakwater as bw
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [2]:
from development_overtopping_DKA import (eurotop2018_6_5, 
                                         surf_similarity, 
                                         gamma_beta_eurotop_2018_6_9, 
                                         calc_beta)
from development_armour_stability_DKA import (vangent_armour_reduction, 
                                              hudson_fixed_slope, 
                                              rock_manual_5_196, 
                                              rock_manual_5_195, 
                                              rock_manual_5_194)

from breakwater.utils.exceptions import user_warning

In [3]:
project_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                             index_col = 1,
                             sheet_name='Input_Project specific')
requirements_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                                  index_col = 0,
                                  sheet_name='Input_requirements')
wave_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx",
                          index_col = 0,
                          sheet_name='input_hydrotechnical',
                         skiprows = 1)
cross_section_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                   sheet_name='Input_Cross section',
                                  skiprows = 1)
concrete_element_data = pd.read_excel(Path("./Input data/") / "test_data_phase_II.xlsx", 
                                      sheet_name='Input_concrete_elements',
                                      index_col = 0,
                                      skiprows = 1)

In [4]:
project_data

,Parameter,Unit,Explanation,Value
Symbol,,,,
rho_w,Water density,kg/m3,NaN,1010
rho_r,Rock density,kg/m3,NaN,2600
rho_c,Concrete density,kg/m3,NaN,2350
g,Gravity,m/s^2,NaN,9.81
ELS,Exploitation limit state,-,NaN,ELS
SLS,Servicability limit state,-,NaN,SLS
ULS,Ultimate limit state,-,NaN,ULS
Cpl_deep,Plunging coefficient deep,-,NaN,6.2
Cs_deep,Surging coefficient deep,-,NaN,1


In [5]:
#requirements_data

In [6]:
wave_data

,Structure,Chainage,Offshore bin,Limit State,wl,Hm0,dir_wave,Tp,"Tm-1,0","Tm0,2"
Calculation_case,,,,,,,,,,
1,ECn2,370N,0,ELS,0.38,1.060,48.1,5.58,3.95,2.78
2,ECn2,370N,30,ELS,0.47,2.060,69.1,8.11,6.66,4.85
3,ECn2,370N,60,ELS,0.56,2.800,78.9,9.01,8.05,6.60
4,ECn2,370N,90,ELS,0.50,2.570,86.5,9.00,7.85,6.30
5,ECn2,370N,120,ELS,0.39,1.650,98.0,7.07,6.02,4.77
...,...,...,...,...,...,...,...,...,...,...
164,ECn2,120N,90,ULS,0.85,1.908,72.7,12.03,11.73,10.95
165,ECn2,120N,120,ULS,0.63,1.584,74.7,9.89,9.49,8.72
166,ECn2,120N,150,ULS,0.65,1.464,78.1,9.62,8.98,7.99


In [7]:
cross_section_data


,Structure,Chainage,dir_structure,z_bed,public,P,tan_a,Gc,slope_foreshore,crest_roughness,cot_a_rear,B,roundhead,safety
0,ECn2,370N,353.4,-6.61,0,0.5,0.666667,1.5,300,rough,1.5,6,No,0
1,ECn2,320N,353.4,-6.52,0,0.5,0.666667,1.5,300,rough,1.5,6,No,0
2,ECn2,270N,353.4,-4.41,0,0.5,0.666667,1.5,300,rough,1.5,6,No,0
3,ECn2,220N,353.4,-3.32,0,0.5,0.666667,1.5,300,rough,1.5,6,No,0
4,ECn2,190N,353.4,-3.00,0,0.5,0.666667,1.5,300,rough,1.5,6,No,0
5,ECn2,170N,353.4,-2.29,1,0.5,0.666667,1.5,300,rough,1.5,6,No,0
6,ECn2,120N,353.4,-1.21,1,0.5,0.666667,1.5,300,rough,1.5,6,No,0


In [21]:
# CALCULATE CREST HEIGHT FOR OVERTOPPING

# Do we want to put this entire part in a function to keep the scripts clean a bit?


Calculation_case = 57
Cross_section_id = 0
# Public access: allowed

# Open project specific parameters
g = project_data.at['g', 'Value']

# Open structure specific parameters
tana          = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure = cross_section_data.at[Cross_section_id, 'dir_structure']
safety        = cross_section_data.at[Cross_section_id, 'safety']

# Get info for sea state. Currently implemented to do only the first sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']

# Get allowed q for the location and calculation case
LS        = wave_data.at[Calculation_case, 'Limit State']
q_allowed = requirements_data.at['Overtopping limit public access', LS]


#Calculate roughness reduction
# Set armour for rougness reduction
armour_layer = 'Rock'
layers       = 2
permeability = 'permeable'

# Calculate surf-similarity with Hm0 and Tm-1,0
xi_m_min_1 = surf_similarity(tana, Hm0, Tm_min_1, g)

gamma_f = bw.core.overtopping.gamma_f(armour_layer, 
                                      xi_m_min_1, 
                                      layers = layers, 
                                      permeability = permeability,
                                      rubble_mound_limit = 'True') # Maximizes gamma_f at 0.6 even for long waves

#Calculate obliqueness reduction. Function based on SAWP-#3504459-V48-IHS-COA-xxx-CAL_Armour_Stability_under_Waves.XLSM
beta = calc_beta(dir_structure, dir_wave)
gamma_beta = gamma_beta_eurotop_2018_6_9(beta)


Rc = eurotop2018_6_5(g, Hm0, q_allowed, gamma_f, gamma_beta, limit = False, safety=safety)
z_crest = Rc + wl

In [22]:
print("Cross-section : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
print("Wave info for : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
print("q_allowed : "+str(q_allowed))
#print("g         : "+str(g))
#print("Hm0       : "+str(Hm0))
#print("Tm_min_1  : "+str(Tm_min_1))
#print("wl        : "+str(wl))
print("xi_m_min_1: "+str(xi_m_min_1))
print("Rc        : "+str(Rc))
print("beta      : "+str(beta))
print("gamma_f   : "+str(gamma_f))
print("gamma_beta: "+str(gamma_beta))
print("z_crest   : "+str(z_crest))


Cross-section : ECn2 - 370N
Wave info for : ECn2 - 370N
q_allowed : 5.0
xi_m_min_1: 3.545428517642052
Rc        : 1.254756265426078
beta      : 65.5
gamma_f   : 0.4
gamma_beta: 0.58735
z_crest   : 1.744756265426078


In [23]:
# CALCULATE REQUIRED STONE DIAMETER (Rock armour)

Calculation_case = 57
Cross_section_id = 0
# Armour = 2 layers rock, permeable core

Storm_duration = 6 #Hours
Safety = 0 #Safety factor on top of regular VDM constants

# Open project specific parameters
g           = project_data.at['g'          , 'Value']
rho_a       = project_data.at['rho_r'      , 'Value']
rho_w       = project_data.at['rho_w'      , 'Value']
Cpl_shallow = project_data.at['Cpl_shallow', 'Value']
Cs_shallow  = project_data.at['Cs_shallow' , 'Value']
c_beta      = project_data.at['c_beta'     , 'Value']
c_rh        = project_data.at['c_rh'       , 'Value']
beta_max    = project_data.at['beta_max'   , 'Value']

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']
Tm_0_2   = wave_data.at[Calculation_case, 'Tm0,2']
os_bin   = wave_data.at[Calculation_case, 'Offshore bin']
N        = np.round(Storm_duration*3600/Tm_0_2)


# Open structure specific parameters
tana            = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure   = cross_section_data.at[Cross_section_id, 'dir_structure']
safety          = cross_section_data.at[Cross_section_id, 'safety']
z_bed           = cross_section_data.at[Cross_section_id, 'z_bed']
slope_foreshore = cross_section_data.at[Cross_section_id, 'slope_foreshore']
P               = cross_section_data.at[Cross_section_id, 'P']
rh              = cross_section_data.at[Cross_section_id, 'roundhead']


# Get allowed Sd for the location and calculation case
LS         = wave_data.at[Calculation_case, 'Limit State']
Sd_allowed = requirements_data.at['Armour damage limit', LS]

#Intermediate calculations
h              = wl-z_bed
Delta          = (rho_a-rho_w)/rho_w
alpha          = np.arctan(tana)
NEN_gradings   = bw.RockGrading(rho=rho_a)
waveinfo       = bw.BattjesGroenendijk(Hm0, h, slope_foreshore)
H2_per         = waveinfo.get_Hp(0.02)
Hs             = waveinfo.get_Hn(3) #NU BATTJES-GROENENDIJK VOOR Hs UIT Hmo. IS DAT WAT WE WILLEN?

#OVERRIDE wave characteristics BECAUSE W+B sheet tales Hs = Hm0
Hs = Hm0
#H2_per = Hs*1.34
#user_warning(f"Hs = Hm0, H2_per taken from W+B calculation due to inconsistencies in Battjes Groenendijk")
xi_s_0_2     = surf_similarity(tana, Hs, Tm_0_2, g)

# Check validity of Van der Meer shallow
vdm_shallow_validity = h/Hs

#Calculate required stone diameter without reduction
Dn50 = bw.core.vandermeer_shallow(Hs, 
                                  H2_per, 
                                  Delta, 
                                  P, 
                                  Sd_allowed, 
                                  N, 
                                  xi_s_0_2, 
                                  alpha, 
                                  Cpl = Cpl_shallow, 
                                  Cs = Cs_shallow, 
                                  safety = Safety)

#Reduce Dn50 with reduction factors from DAR

#Calculate obliqueness reduction. Function based on SAWP-#3504459-V48-IHS-COA-xxx-CAL_Armour_Stability_under_Waves.XLSM
beta = calc_beta(dir_structure, dir_wave)
gamma_beta  = vangent_armour_reduction(beta, c_beta, beta_max)
Dn50_oblique = Dn50*gamma_beta

#Dn50_roundhead, no obliqueness correction on roundhead
Dn50_rh = Dn50*c_rh 

#Select stone size to apply
if rh == "Yes":
    Dn50_selected = Dn50_rh
elif rh == "No":
    Dn50_selected = Dn50_oblique
else:
    user_warning(f"Incorrect input in Roundhead indication")
    Dn50_selected = 10000

# Select grading    
grading = NEN_gradings.get_class(Dn50 = Dn50_selected) 
#It looks like this will only go to the next class if M50>M50_max for a class. Is this the behaviour we want?

In [30]:
#print("Cross-section : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
#print("Wave info for : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
#print("Limit State   : "+str(LS))
#print("Bin           : "+str(os_bin))
#print("Allowed Sd    : "+str(Sd_allowed))
#print("g         : "+str(g))
print("h         : "+str(h))
print("Delta     : "+str(Delta))
print("P         : "+str(P))
#print("Tangent a : "+str(tana))
#print("Hm0       : "+str(Hm0))
print("Hs        : "+str(Hs))
print("H2%       : "+str(H2_per))
print("H2%/Hs    : "+str(H2_per/Hs))
#print("Tm0,2     : "+str(Tm_0_2))
print("N         : "+str(N))
print("xi_s_0_2  : "+str(xi_s_0_2))
print("Dn50      : "+str(Dn50))
print("h/Hs (<3) : "+str(vdm_shallow_validity))
#print("beta      : "+str(beta))
print("gamma_beta: "+str(gamma_beta))
print("Dn50_oblique: "+str(Dn50_oblique))
print("Dn50_rh    : "+str(Dn50_rh))
print("Dn50_select: "+str(round(Dn50_selected,4)))
print("Grading    : "+str(grading))


h         : 7.1000000000000005
Delta     : 1.5742574257425743
P         : 0.5
Hs        : 2.24
H2%       : 2.9997113157614903
H2%/Hs    : 1.3391568373935223
N         : 4821.0
xi_s_0_2  : 2.493488188231773
Dn50      : 0.7964916534126327
h/Hs (<3) : 3.169642857142857
gamma_beta: 0.8559
Dn50_oblique: 0.36780456531402306
Dn50_rh    : 0.8578215107254055
Dn50_select: 0.3678
Grading    : LMA_60/300


In [25]:
concrete_element_data


,Nod,Accropode II-trunk,Accropode II-roundhead,Xbloc-trunk,Xbloc-roundhead
Limit State,,,,,
SLS,0.0,2.8,2.5,2.77,2.57
ULS,0.5,3.1,2.8,3.10,2.90


In [26]:
# Calculate concrete armour stability

Calculation_case = 57
Cross_section_id = 0
unit = 'Xbloc' #Xbloc or Accropode II

# Open project specific parameters
rho_c  = project_data.at['rho_c'      , 'Value'] #For rock armour. Needs adaptation for concrete
rho_w  = project_data.at['rho_w'      , 'Value']

# Get info for sea state
LS     = wave_data.at[Calculation_case, 'Limit State']
Hm0    = wave_data.at[Calculation_case, 'Hm0']

# Open cross-section specific parameters
rh     = cross_section_data.at[Cross_section_id, 'roundhead']

# Intermediate calculations
Hs = Hm0 #TO BE CHANGED DEPENDING ON DESIGN APPROACH
Delta = (rho_c-rho_w)/rho_w


# Create stability case to find outcome for Hudson relationship
if rh == "Yes":
    stability_case = unit+"-roundhead"
elif rh == "No":
    stability_case = unit+"-trunk"
else:
    user_warning(f"Incorrect input in Roundhead identifier")
    stability_case = None

Hudson_outcome = concrete_element_data.at[LS, stability_case]    

Dn50_concrete = hudson_fixed_slope(Hs, Hudson_outcome, Delta)
V_unit = Dn50_concrete**3

In [27]:
#print("Cross-section : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
#print("Wave info for : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
#print("Limit State   : "+str(LS))
#print("rho_c         : "+str(rho_c))
#print("rho_w         : "+str(rho_w))
print("Delta         : "+str(Delta))
print("Roundhead     : "+str(rh))
print("Hs            : "+str(Hs))
print("Hudson outcome: "+str(Hudson_outcome))
print("Dn50_units    : "+str(Dn50_concrete))
print("V_units       : "+str(V_unit))


Delta         : 1.3267326732673268
Roundhead     : No
Hs            : 2.24
Hudson outcome: 2.77
Dn50_units    : 0.6095155989008029
V_units       : 0.22644069233860178


In [28]:
# CALCULATE REAR SIDE ARMOUR STABILITY
# This calculation needs a crest height as input and therefore needs to be done after calculation of 
# the required crest height. If the calculation is done before choosing the final crest height it can give conservative
# results since the crest height in the calculation is lower than the final crest height.
# The calculation can also be performed in the verification stage only.

#To-do
# Hs-Hm0 interpretation
# Select correct Rc
# Adapt grading selection behaviour

Calculation_case = 57
Cross_section_id = 0
# Armour = 2 layers rock, permeable core

Storm_duration = 6 #Hours



# Open project specific parameters
g               = project_data.at['g'              , 'Value']
rho_a           = project_data.at['rho_r'          , 'Value']
slope_roughness = project_data.at['slope_roughness', 'Value']

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
dir_wave = wave_data.at[Calculation_case, 'dir_wave']
Tm_0_2   = wave_data.at[Calculation_case, 'Tm0,2']
Tm_min_1 = wave_data.at[Calculation_case, 'Tm-1,0']

# Open structure specific parameters
tana            = cross_section_data.at[Cross_section_id, 'tan_a']
dir_structure   = cross_section_data.at[Cross_section_id, 'dir_structure']
cota_rear       = cross_section_data.at[Cross_section_id, 'cot_a_rear']
crest_roughness = cross_section_data.at[Cross_section_id, 'crest_roughness']
B               = cross_section_data.at[Cross_section_id, 'B']

# Get allowed Sd for the location and calculation case
LS         = wave_data.at[Calculation_case, 'Limit State']
Sd_allowed = requirements_data.at['Armour damage limit', LS]

#Intermediate calculations
h              = wl-z_bed
beta = calc_beta(dir_structure, dir_wave)
waveinfo       = bw.BattjesGroenendijk(Hm0, h, slope_foreshore)
H2_per         = waveinfo.get_Hp(0.02)
Hs             = waveinfo.get_Hn(3) #NU BATTJES-GROENENDIJK VOOR Hs UIT Hmo. IS DAT WAT WE WILLEN?

#OVERRIDE wave characteristics BECAUSE W+B sheet tales Hs = Hm0
Hs = Hm0 #TO BE ADAPTED IF WE CHOOSE TO DIFFERENTIATE BETWEEN Hs AND Hm0

Delta          = (rho_a-rho_w)/rho_w
xi_s_min_1     = surf_similarity(tana, Hs, Tm_min_1, g)

NEN_gradings   = bw.RockGrading(rho=rho_a)

# Actual calculations
Ru_1_percent, gamma_f, gamma_beta, gamma  = rock_manual_5_196(Hs, xi_s_min_1, slope_roughness, beta)
u_1_percent = rock_manual_5_195(g, slope_roughness, crest_roughness, Ru_1_percent, Rc, B, Hs)
#Rc IS NOW JUST THE PRESENT Rc. THIS NEEDS TO BE COUPLED TO CALCULATION CASE
Dn50_rear = rock_manual_5_194(Sd_allowed, N, u_1_percent, Tm_min_1, Delta, cota_rear, Rc, Hs)

# Select grading    
grading_rear = NEN_gradings.get_class(Dn50 = Dn50_rear) 
#It looks like this will only go to the next class if M50>M50_max for a class. Is this the behaviour we want?

In [29]:
print("Cross-section  : "+str(cross_section_data.at[Cross_section_id,"Structure"])+" - "+str(cross_section_data.at[Cross_section_id,"Chainage"]))
print("Wave info for  : " +str(wave_data.at[Calculation_case,"Structure"])+" - "+str(wave_data.at[Calculation_case,"Chainage"]))      
print("Limit State    : "+str(LS))
print("Hs             : "+str(Hs))
print("xi_s_min_1     : "+str(xi_s_min_1))
print("slope_roughness: "+str(slope_roughness))
print("crest_roughness: "+str(crest_roughness))
print("beta           : "+str(beta))
print("gamma_f        : "+str(gamma_f))
print("gamma_beta     : "+str(gamma_beta))
print("gamma          : "+str(gamma))
print("Ru_1_percent   : "+str(Ru_1_percent))
print("Rc             : "+str(Rc))
print("B              : "+str(B))
print("u_1_percent    : "+str(u_1_percent))
print("Sd_allowed     : "+str(Sd_allowed))
print("N              : "+str(N))
print("Tm_min_1       : "+str(Tm_min_1))
print("Delta          : "+str(Delta))
print("cota_rear:     : "+str(cota_rear))
print("Dn50_rear      : "+str(Dn50_rear))
print("Grading rear   : "+str(grading_rear))

Cross-section  : ECn2 - 370N
Wave info for  : ECn2 - 370N
Limit State    : SLS
Hs             : 2.24
xi_s_min_1     : 3.545428517642052
slope_roughness: rough
crest_roughness: rough
beta           : 65.5
gamma_f        : 0.55
gamma_beta     : 0.8559
gamma          : 0.470745
Ru_1_percent   : 4.044031891608836
Rc             : 1.254756265426078
B              : 6
u_1_percent    : 7.013887880646336
Sd_allowed     : 2.0
N              : 4821.0
Tm_min_1       : 6.37
Delta          : 1.5742574257425743
cota_rear:     : 1.5
Dn50_rear      : 0.5968088741544244
Grading rear   : HMA_300/1000


In [36]:
# CALCULATE TOE STABILITY

Calculation_case = 57
Cross_section_id = 0

# Open project specific parameters
g           = project_data.at['g'          , 'Value']
rho_a       = project_data.at['rho_r'      , 'Value']
rho_w       = project_data.at['rho_w'      , 'Value']

#Get info for sea state
Hm0      = wave_data.at[Calculation_case, 'Hm0']
wl       = wave_data.at[Calculation_case, 'wl']

# Open structure specific parameters
z_bed           = cross_section_data.at[Cross_section_id, 'z_bed']

# Get allowed Sd for the location and calculation case
LS         = wave_data.at[Calculation_case, 'Limit State']
Nod_allowed = requirements_data.at['Toe damage limit', LS]

# Intermediate parameters
h               = wl-z_bed
Delta          = (rho_a-rho_w)/rho_w

#TEMPORARY FIX TO BE IN LINE WITH PHASE II
Hs = Hm0


Dn50_toe = bw.core.toe.toe_stability(Hs, h, ht, Delta, Nod_allowed)

NameError: name 'ht' is not defined

In [37]:
Nod_allowed

0.5